In [1]:
import numpy as np
rng = np.random.default_rng()


In [12]:
A_prev = np.random.randn(2, 5, 7, 4)
W = np.random.randn(3, 3, 4, 8)
b = np.random.randn(1, 1, 1, 8)

In [16]:
# GRADED FUNCTION: zero_pad

def zero_pad(X, pad):
    """
    Pad with zeros all images of the dataset X. The padding is applied to the height and width of an image, 
    as illustrated in Figure 1.
    
    Argument:
    X -- python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
    pad -- integer, amount of padding around each image on vertical and horizontal dimensions
    
    Returns:
    X_pad -- padded image of shape (m, n_H + 2 * pad, n_W + 2 * pad, n_C)
    """
    
    #(≈ 1 line)
    # X_pad = None
    # YOUR CODE STARTS HERE
    X_pad = np.pad(X, ((0, 0), (pad, pad), (pad, pad), (0, 0)))
    
    # YOUR CODE ENDS HERE
    
    return X_pad

In [68]:
# GRADED FUNCTION: conv_single_step

def conv_single_step(a_slice_prev, W, b):
    """
    Apply one filter defined by parameters W on a single slice (a_slice_prev) of the output activation 
    of the previous layer.
    
    Arguments:
    a_slice_prev -- slice of input data of shape (f, f, n_C_prev)
    W -- Weight parameters contained in a window - matrix of shape (f, f, n_C_prev)
    b -- Bias parameters contained in a window - matrix of shape (1, 1, 1)
    
    Returns:
    Z -- a scalar value, the result of convolving the sliding window (W, b) on a slice x of the input data
    """

    #(≈ 3 lines of code)
    # Element-wise product between a_slice_prev and W. Do not add the bias yet.
    # s = None
    # Sum over all entries of the volume s.
    # Z = None
    # Add bias b to Z. Cast b to a float() so that Z results in a scalar value.
    # Z = None
    # YOUR CODE STARTS HERE
    s = a_slice_prev * W
    Z = np.sum(s)
    Z = int((Z + b)[0])
    # YOUR CODE ENDS HERE

    return Z

In [69]:
# GRADED FUNCTION: conv_forward

def conv_forward(A_prev, W, b, hparameters):
    """
    Implements the forward propagation for a convolution function
    
    Arguments:
    A_prev -- output activations of the previous layer, 
        numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    W -- Weights, numpy array of shape (f, f, n_C_prev, n_C)
    b -- Biases, numpy array of shape (1, 1, 1, n_C)
    hparameters -- python dictionary containing "stride" and "pad"
        
    Returns:
    Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward() function
    """
    
    # Retrieve dimensions from A_prev's shape (≈1 line)  
    # (m, n_H_prev, n_W_prev, n_C_prev) = None
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Retrieve dimensions from W's shape (≈1 line)
    # (f, f, n_C_prev, n_C) = None
    (f, f, n_C_prev, n_C) = W.shape
    
    # Retrieve information from "hparameters" (≈2 lines)
    # stride = None
    # pad = None
    stride = hparameters['stride']
    pad = hparameters['pad']
    
    # Compute the dimensions of the CONV output volume using the formula given above. 
    # Hint: use int() to apply the 'floor' operation. (≈2 lines)
    # n_H = None
    # n_W = None
    n_H = int(((n_H_prev - f + 2 * pad) / stride) // 1 + 1)
    n_W = int(((n_W_prev - f + 2 * pad) / stride) // 1 + 1)
    
    # Initialize the output volume Z with zeros. (≈1 line)
    # Z = None
    Z = np.zeros((m, n_H, n_W, n_C))
    
    # Create A_prev_pad by padding A_prev
    # A_prev_pad = None
    A_prev_pad = zero_pad(A_prev, pad)
    
    # YOUR CODE STARTS HERE
    for i in range(m):
        a_prev_pad = A_prev_pad[i]
        for h in range(n_H):
            vert_start = h - (f // 2) + 1 
            vert_end = h + (f // 2) + 2
            for w in range(n_W):
                horiz_start = w - (f // 2) + 1
                horiz_end = w + (f // 2) + 2
                for c in range(n_C):
                    a_slice_prev = a_prev_pad[vert_start : vert_end, horiz_start : horiz_end, :]
                    weights = W[:, :, :, c]
                    biases = b[:, :, :, c]
                    Z[i, h, w, c] = conv_single_step(a_slice_prev, weights, biases)

    
    # YOUR CODE ENDS HERE
    
    # Save information in "cache" for the backprop
    cache = (A_prev, W, b, hparameters)
    
    return Z, cache

In [70]:
A_prev = np.random.randn(2, 5, 7, 4)
W = np.random.randn(3, 3, 4, 8)
b = np.random.randn(1, 1, 1, 8)
hparameters = {"pad" : 1,
               "stride": 2}
Z, cache_conv = conv_forward(A_prev, W, b, hparameters)


C:\Users\Anush\AppData\Local\Temp\ipykernel_22956\1938301191.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Z = int((Z + b)[0])


In [73]:
Z

array([[[[ -3.,   0.,   2.,   0.,   3.,   0.,   0.,  -1.],
         [  1.,   1.,   3.,  -8.,   7.,   2.,   0., -10.],
         [  5.,   0.,   2.,  -5.,   2.,  -1.,  -2.,  -1.],
         [ -3.,  -5.,  -4.,   0.,  -8.,  -7.,  -2.,   0.]],

        [[ -9.,  -7.,   1.,   0.,  -5.,   3.,  -1.,   5.],
         [  4.,   4.,   3.,   1.,   3.,   0.,   2.,   1.],
         [  5.,   0.,  -3.,  -9.,   4.,  -5.,  10.,  -1.],
         [  8.,  -5.,   8.,  -4.,  -2.,   2.,   2.,   1.]],

        [[  5.,   4.,   2.,   2.,   0.,   2.,   6.,  -6.],
         [  3.,  -5.,  -3.,  -9.,  -4.,  -3.,   3.,  14.],
         [  2.,  -4.,   8.,   0.,  -5.,   3.,  -5.,   4.],
         [ -7.,   3.,  -3.,   0., -13.,   1.,  -7.,   6.]]],


       [[[ -4.,  -3.,   5.,   0., -10.,   5.,  -1.,  -2.],
         [ -9.,   2.,   7.,   4.,  -7.,  13.,   9.,  -5.],
         [  2.,   8.,   1.,  -2.,   4.,   5.,  -6.,  -1.],
         [  6.,  -7.,  -6.,  -9.,   1.,  -7.,   0.,  -8.]],

        [[ -6.,  -1.,  -8.,   2.,  -7.,  -1., 

In [71]:
# Retrieve dimensions from A_prev's shape (≈1 line)  
# (m, n_H_prev, n_W_prev, n_C_prev) = None
(m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
print((m, n_H_prev, n_W_prev, n_C_prev) )

(2, 5, 7, 4)


In [54]:

# Retrieve dimensions from W's shape (≈1 line)
# (f, f, n_C_prev, n_C) = None
(f, f, n_C_prev, n_C) = W.shape
print(W.shape)

(3, 3, 4, 8)


In [55]:

# Retrieve information from "hparameters" (≈2 lines)
# stride = None
# pad = None
stride = hparameters['stride']
pad = hparameters['pad']


In [56]:

# Compute the dimensions of the CONV output volume using the formula given above. 
# Hint: use int() to apply the 'floor' operation. (≈2 lines)
# n_H = None
# n_W = None
n_H = int(((n_H_prev - f + 2 * pad) / stride) // 1 + 1)
n_W = int(((n_W_prev - f + 2 * pad) / stride) // 1 + 1)
print(n_H, n_W)

3 4


In [57]:

# Initialize the output volume Z with zeros. (≈1 line)
# Z = None
Z = np.zeros((m, n_H, n_W, n_C))


In [58]:

# Create A_prev_pad by padding A_prev
# A_prev_pad = None
A_prev_pad = zero_pad(A_prev, pad)
A_prev_pad.shape

(2, 7, 9, 4)

In [59]:
A_prev_pad[0].shape

(7, 9, 4)

In [60]:
h = 2
f = 3
vert_start = h - (f // 2) + 1
vert_end = h + (f // 2) + 2
print(vert_start, vert_end)

2 5


In [61]:
f//2

1

In [62]:

# for i in range(None):               # loop over the batch of training examples
    # a_prev_pad = None               # Select ith training example's padded activation
    # for h in range(None):           # loop over vertical axis of the output volume
        # Find the vertical start and end of the current "slice" (≈2 lines)
        # vert_start = None
        # vert_end = None
        
        # for w in range(None):       # loop over horizontal axis of the output volume
            # Find the horizontal start and end of the current "slice" (≈2 lines)
            # horiz_start = None
            # horiz_end = None
            
            # for c in range(None):   # loop over channels (= #filters) of the output volume
                                    
                # Use the corners to define the (3D) slice of a_prev_pad (See Hint above the cell). (≈1 line)
                # a_slice_prev = None
                
                # Convolve the (3D) slice with the correct filter W and bias b, to get back one output neuron. (≈3 line)
                # weights = None
                # biases = None
                # Z[i, h, w, c] = None
# YOUR CODE STARTS HERE


C:\Users\Anush\AppData\Local\Temp\ipykernel_22956\3131362450.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Z[i, h, w, c] = conv_single_step(a_slice_prev, weights, biases)
